In [3]:
import video_transect, misc
from pathlib import Path
from PIL import Image
import exiftool
import datetime
import pandas as pd

In [67]:
# Paths
# im_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/100GOPRO')
im_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO')
csv_file = Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/GnssData/20230620_144519_GnssTimePosHeadCourSpeed.csv')
test_gpkg_output = Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/ImagesGeopackage/20230620_Smola_Skalmen_TestImageGpkg.gpkg')

In [53]:
# Parameters
sample_distance = 1.0 # Units defined by CRS, default is UTM with unit meters

In [54]:
# Get list of image files
im_files = sorted(list(im_dir.glob('*.JPG')))
for im_file in im_files[0:5]:
    print(im_file)
print('...')
print(f'{len(im_files)} image files found')

/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO/G0013750.JPG
/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO/G0013751.JPG
/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO/G0013752.JPG
/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO/G0013753.JPG
/media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO/G0013754.JPG
...
999 image files found


In [55]:
# Try using PIL to get EXIF data - quite limited
im = Image.open(im_files[0])
exif = im.getexif()
creation_time = exif.get(36867)
print(exif)
print(creation_time)

{296: 2, 282: 72.0, 34665: 146, 270: 'DCIM\\102GOPRO\\G0013750.JPG\x00\x00\x00\x00\x00', 271: 'GoPro\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 272: 'HERO10 Black\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 305: 'H21.01.01.46.70\x00\x00\x00\x00\x00\x00\x00\x00', 274: 1, 306: '2023:06:20 16:39:50', 531: 1, 283: 72.0}
None


In [56]:
# Use exiftool to get rich metadata
with exiftool.ExifToolHelper() as et:
    metadata = et.get_metadata(im_files)


In [57]:
for key,value in metadata[0].items():
    print(f'{key}: {value}')

SourceFile: /media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO/G0013750.JPG
ExifTool:ExifToolVersion: 12.4
ExifTool:Warning: [minor] Unrecognized MakerNotes
File:FileName: G0013750.JPG
File:Directory: /media/mha114/Massimal2/Massimal/Smola_Skalmen/OtterTransects/20230620/Images/GoPro1/102GOPRO
File:FileSize: 2702091
File:FileModifyDate: 2024:01:26 11:58:09+01:00
File:FileAccessDate: 2024:02:08 15:48:38+01:00
File:FileInodeChangeDate: 2024:02:08 12:50:59+01:00
File:FilePermissions: 100755
File:FileType: JPEG
File:FileTypeExtension: JPG
File:MIMEType: image/jpeg
File:ExifByteOrder: MM
File:ImageWidth: 5568
File:ImageHeight: 4176
File:EncodingProcess: 0
File:BitsPerSample: 8
File:ColorComponents: 3
File:YCbCrSubSampling: 2 1
EXIF:ImageDescription: DCIM\102GOPRO\G0013750.JPG
EXIF:Make: GoPro
EXIF:Model: HERO10 Black
EXIF:Orientation: 1
EXIF:XResolution: 72
EXIF:YResolution: 72
EXIF:ResolutionUnit: 2
EXIF:Software: H21.01.01.46.70
EXIF:ModifyDate: 

In [58]:
# Show subsecond create date for each file
for d in metadata:
    try:
        print(d['Composite:SubSecCreateDate'])
    except:
        pass # Some images don't have this field(?)

2023:06:20 16:39:50.9440
2023:06:20 16:39:51.4440
2023:06:20 16:39:51.9450
2023:06:20 16:39:52.4450
2023:06:20 16:39:52.9460
2023:06:20 16:39:53.4460
2023:06:20 16:39:53.9470
2023:06:20 16:39:54.4470
2023:06:20 16:39:54.9480
2023:06:20 16:39:55.4480
2023:06:20 16:39:55.9490
2023:06:20 16:39:56.4490
2023:06:20 16:39:56.9500
2023:06:20 16:39:57.4500
2023:06:20 16:39:57.9510
2023:06:20 16:39:58.4510
2023:06:20 16:39:58.9520
2023:06:20 16:39:59.4520
2023:06:20 16:39:59.9530
2023:06:20 16:40:00.4530
2023:06:20 16:40:00.9540
2023:06:20 16:40:01.4540
2023:06:20 16:40:01.9550
2023:06:20 16:40:02.4550
2023:06:20 16:40:02.9560
2023:06:20 16:40:03.4560
2023:06:20 16:40:03.9570
2023:06:20 16:40:04.4570
2023:06:20 16:40:04.9580
2023:06:20 16:40:05.4580
2023:06:20 16:40:05.9590
2023:06:20 16:40:06.4590
2023:06:20 16:40:06.9600
2023:06:20 16:40:07.4600
2023:06:20 16:40:07.9610
2023:06:20 16:40:08.4610
2023:06:20 16:40:08.9620
2023:06:20 16:40:09.4620
2023:06:20 16:40:09.9630
2023:06:20 16:40:10.4630


In [73]:
def get_image_timestamps(image_paths):
    """ Get subsecond-accuracy timestamps for every image file in a list
    
    # Input parameters:
    image_paths:
        List of image paths, used by exiftool method ExifToolHelper.get_metadata()
    
    # Returns:
    image_paths_and_timestamps:
        List of tuples containing (image_path, image_timestamp_string)
    """
    with exiftool.ExifToolHelper() as et:
        metadata = et.get_metadata(image_paths)
    image_paths_and_timestamps = [(im_file.name,md['Composite:SubSecCreateDate']) 
                                  for (im_file,md) in zip(image_paths,metadata) 
                                  if 'Composite:SubSecCreateDate' in md]
    return image_paths_and_timestamps

In [74]:
#image_timestamps = [(im_file,md['Composite:SubSecCreateDate']) for im_file,md in zip(im_files,metadata) if 'Composite:SubSecCreateDate' in md]
image_paths_and_timestamps = get_image_timestamps(im_files)

In [75]:
for im_ts in image_paths_and_timestamps[0:10]:
    print(im_ts)
print(f'Got {len(image_paths_and_timestamps)} valid timestamps.')

('G0013750.JPG', '2023:06:20 16:39:50.9440')
('G0013751.JPG', '2023:06:20 16:39:51.4440')
('G0013752.JPG', '2023:06:20 16:39:51.9450')
('G0013753.JPG', '2023:06:20 16:39:52.4450')
('G0013754.JPG', '2023:06:20 16:39:52.9460')
('G0013755.JPG', '2023:06:20 16:39:53.4460')
('G0013756.JPG', '2023:06:20 16:39:53.9470')
('G0013757.JPG', '2023:06:20 16:39:54.4470')
('G0013758.JPG', '2023:06:20 16:39:54.9480')
('G0013759.JPG', '2023:06:20 16:39:55.4480')
Got 899 valid timestamps.


NOTE: Need to insert more information in image file name(?)

In [61]:
gdf = video_transect.otter_csv_to_geodataframe(csv_file)
#gdf = video_transect.filter_gdf_on_distance(gdf,sample_distance)

In [62]:
# gdf.iloc[500:530]
gdf

,Time,Lat,Lng,geometry
0,2023-06-20 14:45:19.418000+00:00,63.464339,7.771215,POINT (7.77122 63.46434)
1,2023-06-20 14:45:19.513000+00:00,63.464339,7.771212,POINT (7.77121 63.46434)
2,2023-06-20 14:45:19.609000+00:00,63.464339,7.771209,POINT (7.77121 63.46434)
3,2023-06-20 14:45:19.705000+00:00,63.464339,7.771206,POINT (7.77121 63.46434)
4,2023-06-20 14:45:19.817000+00:00,63.464339,7.771203,POINT (7.77120 63.46434)
...,...,...,...,...
145175,2023-06-20 18:47:17.026000+00:00,63.463280,7.775832,POINT (7.77583 63.46328)
145176,2023-06-20 18:47:17.122000+00:00,63.463280,7.775832,POINT (7.77583 63.46328)
145177,2023-06-20 18:47:17.234000+00:00,63.463280,7.775832,POINT (7.77583 63.46328)
145178,2023-06-20 18:47:17.330000+00:00,63.463280,7.775832,POINT (7.77583 63.46328)


In [63]:
# View index for time column in geodataframe
gdf_index = pd.Index(gdf['Time'])
print(gdf_index)

DatetimeIndex(['2023-06-20 14:45:19.418000+00:00',
               '2023-06-20 14:45:19.513000+00:00',
               '2023-06-20 14:45:19.609000+00:00',
               '2023-06-20 14:45:19.705000+00:00',
               '2023-06-20 14:45:19.817000+00:00',
               '2023-06-20 14:45:19.913000+00:00',
               '2023-06-20 14:45:20.009000+00:00',
               '2023-06-20 14:45:20.105000+00:00',
               '2023-06-20 14:45:20.217000+00:00',
               '2023-06-20 14:45:20.314000+00:00',
               ...
               '2023-06-20 18:47:16.530000+00:00',
               '2023-06-20 18:47:16.626000+00:00',
               '2023-06-20 18:47:16.722000+00:00',
               '2023-06-20 18:47:16.834000+00:00',
               '2023-06-20 18:47:16.930000+00:00',
               '2023-06-20 18:47:17.026000+00:00',
               '2023-06-20 18:47:17.122000+00:00',
               '2023-06-20 18:47:17.234000+00:00',
               '2023-06-20 18:47:17.330000+00:00',
            

We can use the method get_indexer to create a new (time-based) index into the geodataframe. The input "nearest" lets us search for the closest matching points in time. 

In [64]:
format = r'%Y:%m:%d %H:%M:%S.%f'
example_datetime_str = r'2023:06:20 16:23:23.4570' #r'2023:08:30 14:35:30.4490'
example_datetime = pd.to_datetime(example_datetime_str,format=format,utc=True)
example_datetime_list = [example_datetime]
closest_time_index = pd.Index(gdf['Time']).get_indexer(example_datetime_list,method='nearest',tolerance=datetime.timedelta(seconds=1))
gdf.iloc[closest_time_index]
#gdf.iloc[gdf_index.get_indexer([example_datetime],method='nearest')]

,Time,Lat,Lng,geometry
58839,2023-06-20 16:23:23.418000+00:00,63.467674,7.774996,POINT (7.77500 63.46767)


In [65]:
def get_otter_image_positions(otter_gdf,image_paths_and_timestamps,
                              dt_format = r'%Y:%m:%d %H:%M:%S.%f',
                              time_tolerance = datetime.timedelta(seconds=1)):
    """ Search Otter geodataframe for positions closest to image (in time)
    
    # Input arguments:
    otter_gdf:      
        GDF created from Otter CSV (see otter_csv_to_geodataframe())
        Only column "Time" (containing UTC datetime objects) is used
    image_paths_and_timestamps:
        List of tuples containing (image_path, image_timestamp_string)
    
    # Returns
    image_gdf:
    
    """
    
    # Extract timestamps and convert to datetime objects
    image_datetimes = [pd.to_datetime(dt,format=dt_format,utc=True) for (_,dt) in image_paths_and_timestamps]

    # Find indices for Otter timestamps that are closest to image timestamps
    closest_time_index = pd.Index(otter_gdf['Time']).get_indexer(image_datetimes,method='nearest',
                                                                 tolerance=time_tolerance)
    
    # Remove images that don't have any matches within the time tolerance
    image_paths = [p for ((p,_),im_index) in zip(image_paths_and_timestamps,closest_time_index)
                   if im_index >= 0] # Images without match get index -1
    closest_time_index = closest_time_index[closest_time_index>=0]
    print(len(closest_time_index))
    assert len(image_paths) == len(closest_time_index)

    # Create copy of Otter gdf which only contains rows corresponding to images
    image_gdf = otter_gdf.iloc[closest_time_index]
    
    # Insert additional column for image path
    image_gdf.insert(image_gdf.shape[1]-1,'ImageFile','')
    # image_gdf.iloc[:,'ImageFile'] = image_paths
    image_gdf.iloc[:,image_gdf.columns.get_loc('ImageFile')] = image_paths
    return image_gdf

In [76]:
image_gdf = get_otter_image_positions(gdf,image_paths_and_timestamps)
image_gdf

899


,Time,Lat,Lng,ImageFile,geometry
68714,2023-06-20 16:39:50.916000+00:00,63.468173,7.768652,G0013750.JPG,POINT (7.76865 63.46817)
68719,2023-06-20 16:39:51.412000+00:00,63.468170,7.768664,G0013751.JPG,POINT (7.76866 63.46817)
68724,2023-06-20 16:39:51.907000+00:00,63.468167,7.768676,G0013752.JPG,POINT (7.76868 63.46817)
68729,2023-06-20 16:39:52.420000+00:00,63.468165,7.768688,G0013753.JPG,POINT (7.76869 63.46816)
68734,2023-06-20 16:39:52.915000+00:00,63.468163,7.768700,G0013754.JPG,POINT (7.76870 63.46816)
...,...,...,...,...,...
73689,2023-06-20 16:48:08.408000+00:00,63.464360,7.774204,G0014744.JPG,POINT (7.77420 63.46436)
73694,2023-06-20 16:48:08.904000+00:00,63.464357,7.774203,G0014745.JPG,POINT (7.77420 63.46436)
73699,2023-06-20 16:48:09.416000+00:00,63.464355,7.774203,G0014746.JPG,POINT (7.77420 63.46436)
73704,2023-06-20 16:48:09.912000+00:00,63.464352,7.774202,G0014747.JPG,POINT (7.77420 63.46435)


In [77]:
image_gdf.to_file(test_gpkg_output, driver="GPKG")